In [5]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys
path="data/"
inputname="human.fasta"
outputname="human_2RFH"
name=outputname

In [2]:
def read_fasta_file(path):
    '''
    used for load fasta data and transformd into numpy.array format
    '''
    fh=open(path)
    seq=[]
    for line in fh:
        if line.startswith('>'):
            continue
        else:
            seq.append(line.replace('\r','').replace('\n',''))
    fh.close()
    matrix_data=np.array([list(e) for e in seq])
    return matrix_data

In [3]:
def fetch_singleline_features(sequence):
    alphabet="AUGC"
    k_num=2
    two_sequence=[]
    for index,data in enumerate(sequence):
        if index <(len(sequence)-k_num+1):
            two_sequence.append("".join(sequence[index:(index+k_num)]))
    parameter=[e for e in itertools.product([0,1],repeat=4)]
    record=[0 for x in range(int(pow(4,k_num)))]
    matrix=["".join(e) for e in itertools.product(alphabet, repeat=k_num)] # AA AU AC AG UU UC ...
    final=[]
    for index,data in enumerate(two_sequence):
        if data in matrix:
            final.extend(parameter[matrix.index(data)])
            record[matrix.index(data)]+=1
            final.append(record[matrix.index(data)]*1.0/(index+1))
    return final

In [4]:
sequences=read_fasta_file(path+inputname)
print "sequences-length:{}".format(len(sequences))
print "single-line-sequence-length:{}".format(len(sequences[0]))
features_data=[]
for index,sequence in enumerate(sequences):
    features_data.append(fetch_singleline_features(sequence))
print "features shape :{}".format(np.array(features_data).shape)
pd.DataFrame(features_data).to_csv(path+outputname+".csv",header=None,index=None)

sequences-length:1450
single-line-sequence-length:41
features shape :(1450, 200)


In [2]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [96]:
"""
    jackknife svm
"""
classifier="SVM"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
X_test=None
Y_test=None
y_predict=[]
y_predict_prob=[]
print "start"
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svm.SVC(), parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X,Y)
C=clf.best_params_['C']
gamma=clf.best_params_['gamma']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    svc = svm.SVC(probability=True,C=C,gamma=gamma) 
    svc.fit(X_train, Y_train)
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    y_p=svc.predict(X_test)
    y_p_p=svc.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'jackknife.xls')
print "end"

start


KeyboardInterrupt: 

In [104]:
"""
    cross validation GBDT
"""
classifier="GBDT"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
X_test=None
Y_test=None
y_predict=[]
y_predict_prob=[]
print "start"
GBDT = GradientBoostingClassifier(learning_rate=0.1,max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(GBDT, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    GBDT = GradientBoostingClassifier(n_estimators=n_estimators,learning_rate=0.1,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt')
    GBDT.fit(X_train,Y_train)
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    y_p=GBDT.predict(X_test)
    y_p_p=GBDT.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'jackknife.xls')
print "end"

start
[ 0.27093405  0.98245994  0.15131244 ...,  0.0853095   0.10106905
  0.16451143]
[[['GBDTn_estimators:118max_depth:5min_samples_split:300min_samples_leaf:60', 0.9097345132743363, 0.9989224137931034, 0.820353982300885, 0.820353982300885, 0.9991150442477876, 0.9053330908154178, 0.9008746355685131, 0.9008746355685131, 0.83288470371853329, 0.9097345132743363, 927.0, 203.0, 1.0, 1129.0, 1130.0, 1130.0]]]
200
end


In [106]:
"""
    cross validation RF
"""
classifier="RF"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
y_predict=[]
y_predict_prob=[]
RF = RandomForestClassifier(max_features='sqrt')
parameters = {'n_estimators':range(10,120,1),'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200),'min_samples_leaf':range(60,101,10)}
clf = GridSearchCV(RF, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
min_samples_split=clf.best_params_['min_samples_split']
min_samples_leaf=clf.best_params_['min_samples_leaf']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    RF = RandomForestClassifier(n_estimators=n_estimators,min_samples_split=min_samples_split,min_samples_leaf=min_samples_leaf,max_depth=max_depth,max_features='sqrt')
    RF.fit(X_train, Y_train)
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    # print clf.best_score_
    y_p=RF.predict(X_test)
    y_p_p=RF.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)

ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"min_samples_split:"+str(min_samples_split)+"min_samples_leaf:"+str(min_samples_leaf),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'jackknife.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[ 0.42148064  0.59639568  0.40840058 ...,  0.40845371  0.40255903
  0.40775773]
[[['RFn_estimators:41max_depth:11min_samples_split:700min_samples_leaf:60', 0.89203539823008848, 1.0, 0.784070796460177, 0.784070796460177, 1.0, 0.8854777221704547, 0.87896825396825395, 0.87896825396825395, 0.80301469820377236, 0.89203539823008848, 886.0, 244.0, 0.0, 1130.0, 1130.0, 1130.0]]]
200


True

In [112]:
"""
    cross validation NB
"""
classifier="NB"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
y_predict=[]
y_predict_prob=[]
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    clf=GaussianNB()
    clf.fit(X_train,Y_train)
    y_p=clf.predict(X_test)
    y_p_p=clf.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier,ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'jackknife.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[  2.46698444e-54   1.00000000e+00   1.48986431e-53 ...,   6.49851476e-55
   2.15731329e-55   2.24911742e-55]
[[['NB', 0.91017699115044248, 1.0, 0.820353982300885, 0.820353982300885, 1.0, 0.9057339467530655, 0.90131259115216333, 0.90131259115216333, 0.8339207731937075, 0.91017699115044248, 927.0, 203.0, 0.0, 1130.0, 1130.0, 1130.0]]]
200


True

In [114]:
"""
    cross validation
"""
classifier="LR"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
y_predict=[]
y_predict_prob=[]
logisReg = LogisticRegression()
C_range = 10.0 ** np.arange(-4,3,1)
grid_parame = dict(C=C_range)
clf = GridSearchCV(logisReg, grid_parame, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X,Y)
C=clf.best_params_['C']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    logisReg = LogisticRegression(C=C)
    logisReg.fit(X_train,Y_train)
    y_p=logisReg.predict(X_test)
    y_p_p=logisReg.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)

ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"C:"+str(C),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'_jackknife.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[ 0.34029302  0.9797358   0.27909121 ...,  0.02465013  0.04939637
  0.10877223]
[[['LRC:1.0', 0.9066371681415929, 0.9914438502673797, 0.820353982300885, 0.820353982300885, 0.9929203539823008, 0.9025221141318283, 0.89782082324455204, 0.89782082324455204, 0.82566099160015194, 0.9066371681415929, 927.0, 203.0, 8.0, 1122.0, 1130.0, 1130.0]]]
200


True

In [115]:
"""
    cross validation
"""
classifier="KNN"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
y_predict=[]
y_predict_prob=[]
knn = KNeighborsClassifier()
#设置k的范围
k_range = list(range(1,10))
leaf_range = list(range(1,2))
weight_options = ['uniform','distance']
algorithm_options = ['auto','ball_tree','kd_tree','brute']
param_gridknn = dict(n_neighbors = k_range,weights = weight_options,algorithm=algorithm_options,leaf_size=leaf_range)
clf = GridSearchCV(knn, param_gridknn, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X,Y)
n_neighbors=clf.best_params_['n_neighbors']
weights=clf.best_params_['weights']
algorithm=clf.best_params_['algorithm']
leaf_size=clf.best_params_['leaf_size']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    knn = KNeighborsClassifier(n_neighbors = n_neighbors,weights = weights,algorithm=algorithm,leaf_size=leaf_size)
    knn.fit(X_train,Y_train)
    y_p=knn.predict(X_test)
    y_p_p=knn.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)

# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_neighbors:"+str(n_neighbors)+"weights:"+str(weights)+"algorithm:"+str(algorithm)+"leaf_size:"+str(leaf_size),
            ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'_jackknife.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[ 0.22450537  0.55600281  0.33165397 ...,  0.33101785  0.33109728
  0.32895952]
[[['KNNn_neighbors:9weights:distancealgorithm:autoleaf_size:1', 0.6566371681415929, 0.8138297872340425, 0.4061946902654867, 0.4061946902654867, 0.9070796460176991, 0.6070015946109916, 0.54191263282172375, 0.54191263282172375, 0.36195183778834733, 0.6566371681415929, 459.0, 671.0, 105.0, 1025.0, 1130.0, 1130.0]]]
200


True

In [116]:
"""
    cross validation XGBoost
"""
classifier="XGBoost"
datapath =path+outputname+".csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
y_predict=[]
y_predict_prob=[]
parameters = [{'n_estimators':range(1,10,1),
                          'max_depth':range(2,10),
                          'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                          'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X_train,Y_train)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']
for index in xrange(len(train_data)):
    X_test=np.array(X[index]).reshape(1,-1)
    Y_test=np.array(Y[index]).reshape(1,-1)
    X_train=None
    Y_train=None
    if index==0:
        X_train=np.array(X[(index+1):])
        Y_train=np.array(Y[(index+1):])
    elif index==2259:
        X_train=np.array(X[0:index])
        Y_train=np.array(Y[0:index])
    else:
        X_train=np.concatenate([np.array(X[0:index]),np.array(X[(index+1):])],axis=0)
        Y_train=np.concatenate([np.array(Y[0:index]),np.array(Y[(index+1):])],axis=0)
    xgboost=XGBClassifier(n_estimators=n_estimators,max_depth=max_depth,learning_rate=learning_rate,subsample=subsample)
    xgboost.fit(X_train,Y_train)
    y_p=xgboost.predict(X_test)
    y_p_p=clf.predict_proba(X_test)
    y_predict.append(y_p)
    y_predict_prob.append(y_p_p)

# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
predict_save=[Y.astype(int),np.array(y_predict).astype(int)[:,0],np.array(y_predict_prob)[:,0,1]]
print np.array(y_predict_prob)[:,0,1]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict_jackknife.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'jackknife.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[ 0.44651958  0.63285792  0.39651224 ...,  0.40390751  0.39574108
  0.40656325]
[[['XGBoostn_estimators:3max_depth:9learning_rate:0.1subsample:0.85', 0.90929203539823011, 0.9989212513484358, 0.8194690265486726, 0.8194690265486726, 0.9991150442477876, 0.9048446456270095, 0.90034030140982013, 0.90034030140982013, 0.83212159137451935, 0.90929203539823011, 926.0, 204.0, 1.0, 1129.0, 1130.0, 1130.0]]]
200


True